In [1]:
import requests
from bs4 import BeautifulSoup
import json

In [2]:
def parse(url):
    html = requests.get(url).text
    soup = BeautifulSoup(html, "html.parser")
    return soup

In [3]:
start = "https://www.yogajournal.com/poses/types/"
soup = parse(start)

In [4]:
len()

TypeError: len() takes exactly one argument (0 given)

In [5]:
l_types = soup.find_all(class_="c-block-standard")

In [6]:
type_pose = l_types[0]
json.loads(type_pose.find(class_='o-heading-link')
           ['data-analytics-data'])['props']

{'domain': '<<analytics_vars.domain>>',
 'path': 'https://www.yogajournal.com/poses/types/arm-balances/',
 'listing_type': 'category',
 'location': 'taxonomy_reference',
 'title': 'Arm Balance Yoga Poses'}

In [28]:
def get_types(soup):
    try:
        l_single_pose = soup.find_all(class_="c-article-lead u-breakout")
        if len(l_single_pose) == 1:
            return [(soup.find(class_="c-article-headings u-spacing u-center-block").find("h1").text, l_single_pose[0].find(class_="lazy")["data-iesrc"], l_single_pose[0].find(class_="lazy")["data-iesrc"])]

        l_types = soup.find_all(
            class_="c-block c-block-standard u-align--center c-block--inline-until-medium c-block--inline--mini")
        if len(l_types) == 0:
            l_types = soup.find_all(
                class_="c-block c-block-standard u-align--center c-block--inline-until-medium")
        l_allPose = []
        for type_pose in l_types:
            dic_data = json.loads(type_pose.find(
                class_='o-heading-link')['data-analytics-data'])['props']
            l_allPose.append((dic_data["title"], dic_data["path"], type_pose.find(
                class_="lazy")["data-iesrc"]))
        return l_allPose
    except:
        return None

In [8]:
soup.find_all(class_="c-article-lead u-breakout")

[]

In [9]:
soup = parse(start)
l_allPose = get_types(soup)
print(l_allPose)

[('Arm Balance Yoga Poses', 'https://www.yogajournal.com/poses/types/arm-balances/', 'https://www.yogajournal.com/wp-content/themes/yogajournal-child/resources/assets/images/recirc-large.jpg'), ('Balancing Yoga Poses', 'https://www.yogajournal.com/poses/types/balancing/', 'https://www.yogajournal.com/wp-content/themes/yogajournal-child/resources/assets/images/recirc-large.jpg'), ('Binding Yoga Poses', 'https://www.yogajournal.com/poses/types/binds/', 'https://www.yogajournal.com/wp-content/themes/yogajournal-child/resources/assets/images/recirc-large.jpg'), ('Chest-Opening Yoga Poses', 'https://www.yogajournal.com/poses/types/chest-openers/', 'https://www.yogajournal.com/wp-content/themes/yogajournal-child/resources/assets/images/recirc-large.jpg'), ('Core Yoga Poses', 'https://www.yogajournal.com/poses/types/core/', 'https://www.yogajournal.com/wp-content/themes/yogajournal-child/resources/assets/images/recirc-large.jpg'), ('Forward Bend Yoga Poses', 'https://www.yogajournal.com/poses

In [94]:
soup = parse(start)
l_allPose = get_types(soup)
all_pose = []
while len(l_allPose) > 0:
    pose_type = l_allPose.pop()
    url = pose_type[1]
    r = requests.get(url)
    if r.url == 'https://www.yogajournal.com/poses/':
        all_pose.append((pose_type[0],pose_type[0].split('|')[-1], pose_type[1],url))
        continue
    soup = BeautifulSoup(r.text, "html.parser")
    l_sub_types = get_types(soup)
    if l_sub_types is None:
        continue
    if len(l_sub_types) == 1:
        all_pose.append((pose_type[0], l_sub_types[0][0], l_sub_types[0][1],url))
        continue
    else:
        if len(l_sub_types) == 1:
            continue
        for sub_type in l_sub_types:
            l_allPose.append(
                (pose_type[0] + '|' + sub_type[0], sub_type[1], sub_type[2],url))

In [46]:
len(set([x[1] for x in all_pose]))

115

In [47]:
pose_classif = {}
for x in all_pose:
    pose_classif[x[1]] = pose_classif.get(x[1], []) + [x[0]]

In [48]:
pose_third = []
for x in all_pose:
    if x[1] != x[0].split('|')[-1]:
        pose_third.append(x)

In [50]:
set([len(x[0].split('|')) for x in all_pose])

{2}

In [95]:
import csv
with open('../data/yogaJournal.csv', 'w', encoding='UTF8', newline='') as f:
    writer = csv.writer(f)
    # write multiple rows
    writer.writerows(all_pose)

In [96]:
!pwd

/Users/yassinelaghrari/code/ylg619/yogAssist/notebooks


In [97]:
import os
import pandas as pd
from os.path import isfile, dirname, join

my_path = "../raw_data/yoga_dataset_links"

from os import listdir
from os.path import isfile, join
onlyfiles = [f"{f}" for f in listdir(my_path) if isfile(join(my_path, f))]

#df = pd.read_csv(onlyfiles[0], delimiter = "\t", header=None)
#df

In [98]:
dic_files = {}
for x in onlyfiles:
    poses = x.replace('.txt','')
    poses = poses.replace('_', ' ')
    poses = poses.split(' or ')
    for pose in poses:
        dic_files[pose.strip().lower().replace(' pose' ,'')] = x

In [99]:
dic_files

{'sitting 1 (normal)': 'Sitting pose 1 (normal).txt',
 'side-reclining leg lift': 'Side-Reclining_Leg_Lift_pose_or_Anantasana_.txt',
 'anantasana': 'Side-Reclining_Leg_Lift_pose_or_Anantasana_.txt',
 'dolphin plank': 'Dolphin_Plank_Pose_or_Makara_Adho_Mukha_Svanasana_.txt',
 'makara adho mukha svanasana': 'Dolphin_Plank_Pose_or_Makara_Adho_Mukha_Svanasana_.txt',
 'pigeon': 'Pigeon_Pose_or_Kapotasana_.txt',
 'kapotasana': 'Pigeon_Pose_or_Kapotasana_.txt',
 'plow': 'Plow_Pose_or_Halasana_.txt',
 'halasana': 'Plow_Pose_or_Halasana_.txt',
 'bridge': 'Bridge_Pose_or_Setu_Bandha_Sarvangasana_.txt',
 'setu bandha sarvangasana': 'Bridge_Pose_or_Setu_Bandha_Sarvangasana_.txt',
 'boat': 'Boat_Pose_or_Paripurna_Navasana_.txt',
 'paripurna navasana': 'Boat_Pose_or_Paripurna_Navasana_.txt',
 'scorpion': 'Scorpion_pose_or_vrischikasana.txt',
 'vrischikasana': 'Scorpion_pose_or_vrischikasana.txt',
 'noose': 'Noose_Pose_or_Pasasana_.txt',
 'pasasana': 'Noose_Pose_or_Pasasana_.txt',
 'garland': 'Garlan

In [100]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [101]:
df = pd.DataFrame(all_pose)
df['file_link'] = df[1].apply(lambda x: dic_files[x.strip().lower().replace(' pose' ,'')] if x.strip().lower().replace(' pose' ,'') in dic_files else '')

In [106]:
df[df['file_link']=='']

,0,1,2,3,file_link
0,Yoga Mudras|Wheel of Dharma Seal,Wheel of Dharma Seal,https://www.yogajournal.com/wp-content/uploads...,https://www.yogajournal.com/poses/dharmachakra...,
1,Yoga Mudras|Upright Seal,Upright Seal,https://www.yogajournal.com/wp-content/uploads...,https://www.yogajournal.com/poses/linga-mudra/,
2,Yoga Mudras|Unshakeable Trust Seal,Unshakeable Trust Seal,https://www.yogajournal.com/wp-content/uploads...,https://www.yogajournal.com/poses/vajrapradama...,
3,Yoga Mudras|Truth Seal,Truth Seal,https://www.yogajournal.com/wp-content/uploads...,https://www.yogajournal.com/poses/tattva-mudra/,
4,Yoga Mudras|Salutation Seal,Salutation Seal,https://www.yogajournal.com/wp-content/uploads...,https://www.yogajournal.com/poses/salutation-s...,
5,Yoga Mudras|Meditation Seal,Meditation Seal,https://www.yogajournal.com/wp-content/uploads...,https://www.yogajournal.com/poses/dhyana-mudra/,
6,Yoga Mudras|Lotus Seal,Lotus Seal,https://www.yogajournal.com/wp-content/uploads...,https://www.yogajournal.com/poses/padma-mudra/,
7,Yoga Mudras|Life Force Seal,Life Force Seal,https://www.yogajournal.com/wp-content/uploads...,https://www.yogajournal.com/poses/prana-mudra/,
8,Yoga Mudras|Kali Mudra,Kali Mudra,https://www.yogajournal.com/wp-content/uploads...,https://www.yogajournal.com/poses/kali-mudra/,
9,Yoga Mudras|Ganesha Mudra,Ganesha Mudra,https://www.yogajournal.com/wp-content/uploads...,https://www.yogajournal.com/poses/ganesha-mudra/,


In [92]:
df[2][0]

'https://www.yogajournal.com/wp-content/uploads/2017/03/dana-trixie-flynn-in-dharma-chakra-mudra.jpg?crop=535:301&width=1070&enable=upscale'

In [118]:
#dico_map = {
#    'Plank Pose – Step by Step Instructions' : 'Plank Pose'
#}
#df['1_remake'] = df[1].map(dico_map, 'ignore')


In [119]:
df

,0,1,2,3,file_link,1_remake
0,Yoga Mudras|Wheel of Dharma Seal,Wheel of Dharma Seal,https://www.yogajournal.com/wp-content/uploads...,https://www.yogajournal.com/poses/dharmachakra...,,NaN
1,Yoga Mudras|Upright Seal,Upright Seal,https://www.yogajournal.com/wp-content/uploads...,https://www.yogajournal.com/poses/linga-mudra/,,NaN
2,Yoga Mudras|Unshakeable Trust Seal,Unshakeable Trust Seal,https://www.yogajournal.com/wp-content/uploads...,https://www.yogajournal.com/poses/vajrapradama...,,NaN
3,Yoga Mudras|Truth Seal,Truth Seal,https://www.yogajournal.com/wp-content/uploads...,https://www.yogajournal.com/poses/tattva-mudra/,,NaN
4,Yoga Mudras|Salutation Seal,Salutation Seal,https://www.yogajournal.com/wp-content/uploads...,https://www.yogajournal.com/poses/salutation-s...,,NaN
5,Yoga Mudras|Meditation Seal,Meditation Seal,https://www.yogajournal.com/wp-content/uploads...,https://www.yogajournal.com/poses/dhyana-mudra/,,NaN
6,Yoga Mudras|Lotus Seal,Lotus Seal,https://www.yogajournal.com/wp-content/uploads...,https://www.yogajournal.com/poses/padma-mudra/,,NaN
7,Yoga Mudras|Life Force Seal,Life Force Seal,https://www.yogajournal.com/wp-content/uploads...,https://www.yogajournal.com/poses/prana-mudra/,,NaN
8,Yoga Mudras|Kali Mudra,Kali Mudra,https://www.yogajournal.com/wp-content/uploads...,https://www.yogajournal.com/poses/kali-mudra/,,NaN
9,Yoga Mudras|Ganesha Mudra,Ganesha Mudra,https://www.yogajournal.com/wp-content/uploads...,https://www.yogajournal.com/poses/ganesha-mudra/,,NaN
